In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train_data = pd.read_csv("./train.csv")
train_data.head()

test_data = pd.read_csv("./test.csv")


./SpaceshipTitanic.ipynb
./test.csv
./train.csv


In [31]:
test_data.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,PassengerGroup,PassengerNumber,Deck,DeckNumber,DeckSide
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,0013,01,G,3,S
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0018,01,F,4,S
2,0019_01,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0019,01,C,0,S
3,0021_01,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0021,01,C,1,S
4,0023_01,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,0023,01,F,5,S


In [29]:
# Test Data parsing
Seperate_Passenger_ID = test_data["PassengerId"].str.split("_", n=1, expand=True)
test_data["PassengerGroup"]=Seperate_Passenger_ID[0]
test_data["PassengerNumber"]=Seperate_Passenger_ID[1]

Seperate_Cabin=test_data["Cabin"].str.split("/", n=2, expand=True)
test_data["Deck"]=Seperate_Cabin[0]
test_data["DeckNumber"]=Seperate_Cabin[1]
test_data["DeckSide"]=Seperate_Cabin[2]
test_data.drop(columns=["Cabin"], inplace=True)


In [30]:
# Train Data parsing

Seperate_Passenger_ID = train_data["PassengerId"].str.split("_", n=1, expand=True)
train_data["PassengerGroup"]=Seperate_Passenger_ID[0]
train_data["PassengerNumber"]=Seperate_Passenger_ID[1]

Seperate_Cabin=train_data["Cabin"].str.split("/", n=2, expand=True)
train_data["Deck"]=Seperate_Cabin[0]
train_data["DeckNumber"]=Seperate_Cabin[1]
train_data["DeckSide"]=Seperate_Cabin[2]
train_data.drop(columns=["Cabin"], inplace=True)

In [32]:
train_data['DeckNumber'] = train_data['DeckNumber'].astype('Int64')
train_data['PassengerGroup'] = train_data['PassengerGroup'].astype('Int64')
test_data['DeckNumber'] = test_data['DeckNumber'].astype('Int64')
test_data['PassengerGroup'] = test_data['PassengerGroup'].astype('Int64')

print(train_data.dtypes)

# train_data = train_data.astype({'PassengerGroup': int, 'PassengerNumber': int, 'DeckNumber': int})
# test_data = test_data.astype({'PassengerGroup': int, 'PassengerNumber': int, 'DeckNumber': int})


PassengerId         object
HomePlanet          object
CryoSleep           object
Destination         object
Age                float64
VIP                 object
RoomService        float64
FoodCourt          float64
ShoppingMall       float64
Spa                float64
VRDeck             float64
Name                object
Transported           bool
PassengerGroup       Int64
PassengerNumber     object
Deck                object
DeckNumber           Int64
DeckSide            object
dtype: object


In [33]:
print(train_data.columns)

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name',
       'Transported', 'PassengerGroup', 'PassengerNumber', 'Deck',
       'DeckNumber', 'DeckSide'],
      dtype='object')


In [54]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Transported"]

features = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','PassengerGroup', 'PassengerNumber', 'Deck', 'DeckNumber', 'DeckSide']
X = pd.get_dummies(train_data[features])
print(X)
X_test = pd.get_dummies(test_data[features])
print(X_test)
numberOfTrees=200
maximumDepth=10
model = RandomForestClassifier(n_estimators=numberOfTrees, max_depth=maximumDepth, random_state=1)
model.fit(X, y)

       Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
0     39.0          0.0        0.0           0.0     0.0     0.0   
1     24.0        109.0        9.0          25.0   549.0    44.0   
2     58.0         43.0     3576.0           0.0  6715.0    49.0   
3     33.0          0.0     1283.0         371.0  3329.0   193.0   
4     16.0        303.0       70.0         151.0   565.0     2.0   
...    ...          ...        ...           ...     ...     ...   
8688  41.0          0.0     6819.0           0.0  1643.0    74.0   
8689  18.0          0.0        0.0           0.0     0.0     0.0   
8690  26.0          0.0        0.0        1872.0     1.0     0.0   
8691  32.0          0.0     1049.0           0.0   353.0  3235.0   
8692  44.0        126.0     4688.0           0.0     0.0    12.0   

      PassengerGroup  DeckNumber  HomePlanet_Earth  HomePlanet_Europa  ...  \
0                  1           0             False               True  ...   
1                  2       

RandomForestClassifier(max_depth=10, n_estimators=200, random_state=1)

In [53]:
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.loc[:,"PassengerId"], 'Transported': predictions})
output.to_csv('submission200Trees10MaxDepth.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [55]:
train_data_answers=train_data["Transported"]

train_data_predictions=model.predict(X)
train_data_predictions_df = pd.DataFrame({'PassengerId': train_data.loc[:,"PassengerId"], 'Transported': train_data_predictions})


total=0
correct=0

for i in range(len(train_data_predictions_df)):
    predictedRow=train_data_predictions_df.iloc[i]
    predictedId=predictedRow["PassengerId"]
    predictedDiagnosis=predictedRow["Transported"]

    trainRow=train_data.iloc[i]
    trainId=trainRow["PassengerId"]
    trainDiagnosis=trainRow["Transported"]
    print(type(trainId))
    print(type(predictedId))

    if trainId!=predictedId:
        print("not same id at row "+str(i))
    else:
        total=total+1
        if predictedDiagnosis==trainDiagnosis:
            correct=correct+1

print("Total rows: "+str(total)+"\nCorrect diagnosis: "+str(correct)+"\nPercentage correct: "+str(correct/total))


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [ ]:
submission = pd.read_csv("./submission.csv")
submission.head()